# Assignment 3

<font color=magenta>__AROUND THE WORLD IN EIGHTY DAYS__</font>

Analysis of visited cities

*zuzan*

In [1]:
from geotext import GeoText
import requests
import re
from  bs4 import BeautifulSoup
import time
from geopy.geocoders import Nominatim
import folium
import numpy as np

#### 1. Getting the book text devided into paragraphs

In [2]:
r = requests.get('http://www.gutenberg.org/files/103/103-h/103-h.htm')
soup = BeautifulSoup(r.content, 'html.parser')
book = soup.find_all('p')

#### 2. Since I will use GeoText, I need to change names of some cities

In [3]:
def change_names(par):
    par = par.replace("Bombay", "Mumbai")
    par = par.replace("Calcutta", "Kolkata")
    par = par.replace("Hong Kong", "Victoria")
    par = par.replace('Medicine Bow', 'Wyoming')
    par = par.replace('Fort Kearney', 'Minden')
    #par = par.replace('Queenstown', 'Cobh')   I should change this name here, but geotext does not know city 'Cobh'
    par = par.replace("Stuart","GuysName")   # The travellers names could be considered as cities' names, so I change them
    par = par.replace("Forster","GuysName")
    par = par.replace("Wilson","GuysName")
    par = par.replace("Reno","GuysName")
    return par

#### 3. Getting list of all visited cities, basing on key words in a sentence

In [5]:
key_words = ['from','arrived','left','reached','visited','passing','departed',' abandoned','stayed','stopped','arrival','Reached','Left','reach']
   # key words around a city name can suggest a visit there

visited_cities = []

for paragraph in book:
    text = paragraph.text.replace("\r\n"," ").replace("'"," ").replace('"','').replace('—',' ')  # removing a few characters
    #print(text)
    text = change_names(text)
    #print(text)
    sentences = re.split(r'[.|;|!|?]', text)   # how should the paragraph be split into sentences

    for sentence in sentences:             # iterating along sentences
        places = GeoText(sentence)         # using GeoText to find cities in a sentence
        names_cities = places.cities
        #print(sentence)                    # controlling printing
        #print(names_cities)
        if len(names_cities) > 0:
            words = sentence.replace(",","").replace(":","").split(" ")   # removing other characters from a sentence and split it into words by a space
            correct_words = [x for x in words if x != '']
            #print(correct_words)
            for word in correct_words:     # if there was any city mentioned in the sentence, they are included in visited_cities list, if any of key words appears in the sentence
                if word in key_words:
                    print(names_cities)
                    visited_cities = visited_cities + names_cities
                    break


['London']
['London']
['London']
['London', 'West End']
['Suez']
['Suez', 'London']
['Brindisi']
['Brindisi']
['Suez', 'Aden']
['Mumbai']
['Suez', 'Asia']
['London']
['London']
['London']
['Paris', 'Brindisi', 'Suez', 'Mumbai', 'Kolkata', 'Victoria', 'Yokohama', 'San Francisco', 'New York', 'London']
['Suez']
['London']
['London']
['London']
['Brindisi', 'Mumbai', 'Kolkata', 'Mumbai']
['Mumbai']
['Aden']
['Mumbai']
['London']
['Mumbai', 'Kolkata']
['Mumbai', 'Kolkata']
['Kolkata']
['Mumbai', 'London']
['Mumbai']
['Mumbai']
['London']
['Mumbai']
['Mumbai', 'Kolkata']
['Kolkata']
['Kolkata']
['Kolkata', 'Victoria']
['Victoria']
['Kolkata', 'Victoria']
['Kolkata']
['Mumbai']
['Kolkata', 'Mumbai', 'Suez']
['Kolkata', 'Victoria']
['Victoria']
['Yokohama']
['Victoria']
['Mumbai', 'Victoria']
['Kolkata']
['Malacca']
['Victoria']
['Victoria']
['Victoria']
['Yokohama']
['Yokohama', 'San Francisco', 'Victoria', 'Yokohama']
['Holland']
['Victoria']
['Victoria']
['Yokohama']
['Yokohama', 'Victoria

In [6]:
visited_cities = sorted(set(visited_cities))

In [7]:
visited_cities[visited_cities.index('Victoria')] = 'Hong Kong'   # Changing back Victoria to Hong Kong (while creating map, Victoria is a city in Australia)
visited_cities[visited_cities.index('Queenstown')] = 'Cobh'      # Queenstown is also in New Zealand, so we should use other name of Ireland's Queenstown
visited_cities[visited_cities.index('Minden')] = 'Fort Kearney'  # Changing back to Fort Kearney, because Minden is a city in Germany

In [8]:
print(len(visited_cities))  # how many cities I found
print(visited_cities)

45
['Aden', 'Asia', 'Auburn', 'Bay', 'Bordeaux', 'Brindisi', 'Cedar', 'Chicago', 'Columbus', 'Dublin', 'Durant', 'Formosa', 'Fort Wayne', 'Fremont', 'Golden Gate', 'Holland', 'Kearney', 'Kolkata', 'Laramie', 'Liverpool', 'London', 'Malacca', 'Fort Kearney', 'Missouri', 'Mumbai', 'New York', 'North Platte', 'Oakland', 'Ogden', 'Omaha', 'Paris', 'Pittsburgh', 'Cobh', 'Sacramento', 'Salt Lake', 'San Francisco', 'San Pablo', 'Shanghai', 'Sioux', 'Southampton', 'Suez', 'Hong Kong', 'West End', 'Wyoming', 'Yokohama']


Actually visited cities according to https://en.wikivoyage.org/wiki/Around_the_World_in_Eighty_Days:

In [9]:
actual_cities = ['London', 'Paris', 'Turin', 'Brindisi', 'Suez', 'Aden', 'Mumbai', 'Allahabad', 'Kolkata', 
                 'Singapore', 'Hong Kong', 'Shanghai', 'Yokohama', 'San Francisco', 'Salt Lake', 'Wyoming', 
                 'Fort Kearney', 'Omaha', 'Chicago', 'New York', 'Cobh', 'Dublin', 'Liverpool']

How those lists differ from each other?

In [10]:
set(visited_cities) - set(actual_cities)    # probably wrongly chosen cities

{'Asia',
 'Auburn',
 'Bay',
 'Bordeaux',
 'Cedar',
 'Columbus',
 'Durant',
 'Formosa',
 'Fort Wayne',
 'Fremont',
 'Golden Gate',
 'Holland',
 'Kearney',
 'Laramie',
 'Malacca',
 'Missouri',
 'North Platte',
 'Oakland',
 'Ogden',
 'Pittsburgh',
 'Sacramento',
 'San Pablo',
 'Sioux',
 'Southampton',
 'West End'}

In [11]:
set(actual_cities) - set(visited_cities)   # cities which I should have found, but I didn't

{'Allahabad', 'Singapore', 'Turin'}

Why didn't I find those three cities?

In [12]:
GeoText('Allahabad, Singapore, Turin.').cities 

['Turin']

GeoText can't find two of them (Allahabad, Singapore) and I can't find any documentation of GeoText how to replace their names. 

But what's wrong with Turin? <br />
    Checking in the book, it appears only in the following sentence:<br />
    *' "Reached Turin by Mont Cenis, Friday, October 4th, at 6.35 a.m. "Left Turin, Friday, at 7.20 a.m. '*

In [13]:
GeoText('Reached Turin by Mont Cenis, Friday, October 4th, at 6').cities  

[]

In [14]:
GeoText('reached Turin by Mont Cenis, Friday, October 4th, at 6').cities

['Turin']

The problem is the word before 'Turin' starting with capital letter. It causes wrong classification in GeoText. Probably using any dataset with world's cities' names will be more efficient than using GeoText.

#### 4. Locating the cities on a map

Firstly, my 45 cities.

In [15]:
geolocator = Nominatim()
positions = dict()

for city in visited_cities:
    while True:
        try: 
            position = geolocator.geocode(city)
        except:
            time.sleep(5)
            continue
        break
    if position:
        location=[position.latitude, position.longitude]
        positions.update({city:location})
        del position 
    else:
        print("Could not get position for {}".format(city))

print(positions)

C:\Users\zuzan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


{'Aden': [12.833333, 44.916667], 'Asia': [51.2086975, 89.2343748], 'Auburn': [32.535699, -85.48677785023041], 'Bay': [47.2885, 5.71839], 'Bordeaux': [44.841225, -0.5800364], 'Brindisi': [40.6362271, 17.688500723698695], 'Cedar': [41.7608608, -91.1263198], 'Chicago': [41.8755616, -87.6244212], 'Columbus': [39.9622601, -83.0007065], 'Dublin': [53.3497645, -6.2602732], 'Durant': [41.5997498, -90.9107038], 'Formosa': [23.9739374, 120.9820179], 'Fort Wayne': [41.0799898, -85.1386015], 'Fremont': [37.5482697, -121.9885719], 'Golden Gate': [37.8303213, -122.4797496], 'Holland': [52.5001698, 5.7480821], 'Kearney': [40.4906216, -98.9472344], 'Kolkata': [22.54541245, 88.3567751581234], 'Laramie': [41.311367, -105.591101], 'Liverpool': [53.407154, -2.991665], 'London': [51.5073219, -0.1276474], 'Malacca': [2.2245111, 102.2614662], 'Fort Kearney': [40.771124, -98.3975633], 'Missouri': [38.7604815, -92.5617875], 'Mumbai': [18.9387711, 72.8353355], 'New York': [40.7127281, -74.0060152], 'North Platt

In [16]:
mapa = folium.Map(location=[0,0], zoom_start=2)

for city, location in positions.items():
    mapa.add_child(folium.Marker(location=location, popup=folium.Popup(city), icon=folium.Icon(color='lightred',icon='pushpin')))
    
locations = [item[1] for item in positions.items()]
loc_sort = sorted(locations, key = lambda x: x[1])
#loc_sort = loc_sort[23:] + loc_sort[:23]  # the joyrney started in London
folium.PolyLine(loc_sort, color="red", weight=2.5, opacity=1).add_to(mapa)
  
mapa

Actually visited cities.

In [17]:
geolocator = Nominatim()
positions1 = dict()

for city in actual_cities:
    while True:
        try: 
            position = geolocator.geocode(city)
        except:
            time.sleep(5)
            continue
        break
    if position:
        location=[position.latitude, position.longitude]
        positions1.update({city:location})
        del position 
    else:
        print("Could not get position for {}".format(city))

print(positions1)

C:\Users\zuzan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


{'London': [51.5073219, -0.1276474], 'Paris': [48.8566969, 2.3514616], 'Turin': [45.0677551, 7.6824892], 'Brindisi': [40.6362271, 17.688500723698695], 'Suez': [29.974498, 32.537086], 'Aden': [12.833333, 44.916667], 'Mumbai': [18.9387711, 72.8353355], 'Allahabad': [25.4381302, 81.8338005], 'Kolkata': [22.54541245, 88.3567751581234], 'Singapore': [1.357107, 103.8194992], 'Hong Kong': [22.2793278, 114.1628131], 'Shanghai': [31.2322758, 121.4692071], 'Yokohama': [35.444991, 139.636768], 'San Francisco': [37.7790262, -122.4199061], 'Salt Lake': [40.6632297, -111.9103124], 'Wyoming': [43.1700264, -107.5685348], 'Fort Kearney': [40.771124, -98.3975633], 'Omaha': [41.2587459, -95.9383758], 'Chicago': [41.8755616, -87.6244212], 'New York': [40.7127281, -74.0060152], 'Cobh': [51.8500947, -8.294279], 'Dublin': [53.3497645, -6.2602732], 'Liverpool': [53.407154, -2.991665]}


In [18]:
mapa = folium.Map(location=[0,0], zoom_start=2)

for city, location in positions1.items():
    mapa.add_child(folium.Marker(location=location, popup=folium.Popup(city), icon=folium.Icon(color='lightgreen',icon='ok')))
    
locations = [item[1] for item in positions1.items()]
loc_sort = sorted(locations, key = lambda x: x[1])
#loc_sort = loc_sort[23:] + loc_sort[:23] 
folium.PolyLine(loc_sort, color="green", weight=2.5, opacity=1).add_to(mapa)
  
mapa

# I wanted the marker to show the order of visiting cities (starting from London with 1), 
# but I couldnt find any idea in the internet :(

Let's see, which cities shouldn't I include.

In [20]:
wrong_cities = list(set(visited_cities) - set(actual_cities))

geolocator = Nominatim()
positions1 = dict()

for city in wrong_cities:
    while True:
        try: 
            position = geolocator.geocode(city)
        except:
            time.sleep(5)
            continue
        break
    if position:
        location=[position.latitude, position.longitude]
        positions1.update({city:location})
        del position 
    else:
        print("Could not get position for {}".format(city))

print(positions1)

C:\Users\zuzan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


{'Asia': [51.2086975, 89.2343748], 'Kearney': [40.4906216, -98.9472344], 'Golden Gate': [37.8303213, -122.4797496], 'Ogden': [41.2230048, -111.9738429], 'Durant': [41.5997498, -90.9107038], 'Bordeaux': [44.841225, -0.5800364], 'Fort Wayne': [41.0799898, -85.1386015], 'Southampton': [50.9025349, -1.404189], 'Oakland': [37.8044557, -122.2713563], 'Sioux': [43.0574079, -96.1602368], 'Bay': [47.2885, 5.71839], 'Missouri': [38.7604815, -92.5617875], 'Sacramento': [38.5815719, -121.4943996], 'North Platte': [41.1238873, -100.7654232], 'Fremont': [37.5482697, -121.9885719], 'Pittsburgh': [40.4416941, -79.9900861], 'San Pablo': [-23.5506507, -46.6333824], 'Auburn': [32.535699, -85.48677785023041], 'Laramie': [41.311367, -105.591101], 'West End': [42.3639186, -71.0638993], 'Columbus': [39.9622601, -83.0007065], 'Holland': [52.5001698, 5.7480821], 'Formosa': [23.9739374, 120.9820179], 'Malacca': [2.2245111, 102.2614662], 'Cedar': [41.7608608, -91.1263198]}


In [21]:
mapa = folium.Map(location=[0,0], zoom_start=2)

for city, location in positions1.items():
    mapa.add_child(folium.Marker(location=location, popup=folium.Popup(city), icon=folium.Icon(color='orange',icon='ok'))) 
mapa

Examples of wrong classified cities with their placement in the book:

ex 1. San Pablo, Brazil & Bay, Finland:

    "The line from San Francisco to Sacramento runs in a north-easterly direction, along the American River, which empties into San Pablo Bay."
    
    
ex 2. Bordeaux, France:

    "You must remember that, since we started, we have kept up hot fires in all our furnaces, and, though we had coal enough to go on short steam from New York to Bordeaux, we haven t enough to go with all steam from New York to Liverpool."
    
    
ex 3. Pittsburgh, USA:

    "Fogg passed at once from one to the other, and the locomotive of the Pittsburgh, Fort Wayne, and Chicago Railway left at full speed, as if it fully comprehended that that gentleman had no time to lose."
    
    
ex 4. Formosa - old name of Taiwan:
    
    "The Tankadere entered the Straits of Fo-Kien, which separate the island of Formosa from the Chinese coast, in the small hours of the night, and crossed the Tropic of Cancer."
    
    
ex 5. Malacca, Malaysia:

    "During the afternoon of Wednesday, 30th October, the Rangoon entered the Strait of Malacca, which separates the peninsula of that name from Sumatra."
    
    
ex 6. Holland:
    
    "Meeting a broker, he made the inquiry, to learn that Jeejeeh had left China two years before, and, retiring from business with an immense fortune, had taken up his residence in Europe in Holland the broker thought, with the merchants of which country he had principally traded."
    
   *This place appeard a few times in a book, but only once was clasified as visited. Maybe deleting the cities which occured only once in my appending list will solve this problem.*
    
    
ex 7. West End, Australia:
       
    "Phileas Fogg rightly suspected that his departure from London would create a lively sensation at the West End."
    
   *Maybe the idea mentioned in the previous point will solve it too, this city appeared only once in the whole book.*
    
    
ex 8. Southampton, GB:

    " the Hamburg boats did not go directly to Liverpool or London, but to Havre; and the additional trip from Havre to Southampton would render Phileas Fogg's last efforts of no avail."
    
   *Also appeared only once in the whole book.*
   
   
To sum up, it might be possible that the travellers stopped in some of the other cities, but to be sure it would be nice to know the whole book and the context. If I wanted to perform further analysis, I would try first excluding cities that appeared only once, then using my own list of the cities in the world instead of GeoText, and then somehow upgrade the method of checking the key words in a sentence.